# Setup

In [1]:
import pandas as pd
import copy
import numpy as np
import warnings
import math

from pprint import pformat
import xlsxwriter
import os
import json
import pprint
from datetime import datetime

# Import functions

In [2]:
def read_excel_file(filepath, sheet_names):
    print("Apply strategies: read excel-file ...")
    xl = pd.ExcelFile(filepath)
    dfs = {sheet_name: xl.parse(sheet_name).T.to_dict() for sheet_name in sheet_names}
    
    return tuple(dfs.values())

In [3]:
def unzip_dict(df):
    df_new = [df[a] for a in df]
    return df_new

## D_FSE

In [30]:
def add_träger_column(df):
    print("Apply strategies - NP: add täger-column ...")
    for a in df:
        #lowest_e = True if a+1 in df and df[a+1]["Ebene"]<= df[a]["Ebene"] else False
        current_e=a["Ebene"]
        current_nr=a["Nr."]
        e0, e1, e2, e3, e4 = (
            current_nr if current_e == 0 else e0 if current_e >= 0 else "-", 
            current_nr if current_e == 1 else e1 if current_e >= 1 else "-",
            current_nr if current_e == 2 else e2 if current_e >= 2 else "-",
            current_nr if current_e == 3 else e3 if current_e >= 3 else "-",
            current_nr if current_e == 4 else "-"
        )
        upper_e = (
            e3 if current_e == 4
            else e2 if current_e == 3
            else e1 if current_e == 2
            else e0 if current_e == 1
            else "-"
        )
        a["Träger"]=upper_e
    
    return df

In [31]:
def transform_excel_sheet_to_matrix(df):
    print("Apply strategies - NP: transform excel-sheet to D_FSE matrix ...")

    df_new = [{
        "name": PSE['Bezeichnung'],
        "reference product": PSE['Bezeichnung'],
        "unit": "exc.",
        "code": PSE['Nr.'],
        "level": PSE['Ebene'],
        "exchanges": [{
            "name": PSE['Bezeichnung'],
            "reference product": PSE['Bezeichnung'],
            "unit": "pcs.",
            "code": PSE['Nr.'],
            "type": "production",
            "amount": 1
        }]}
        for PSE in df]

    for a2 in df:
        for a_count, PSE in enumerate(df_new):
            if a2['Träger'] == PSE["code"]:
                PSE["exchanges"].append({
                    "name": a2['Bezeichnung'],
                    "reference product": a2['Bezeichnung'],
                    "unit": "exc.",
                    "code": a2['Nr.'],
                    "type": "technosphere",
                    "amount": -1
                })

    return df_new

In [32]:
def sort_column_by_level(matrix):
    print("Apply strategies - NP: sort matrix-column by level (descending) ...")
    
    matrix_new=sorted(matrix, key=lambda d: d['level'])
    
    return matrix_new

## D_NPZ

In [33]:
def filter_out_NPZ_PZ(df):
    print("Apply strategies - NP: create '4.2' Tab.")
    df_NPZ = [df[c] for c in df if df[c]["Nr."].startswith("NP")]
    return df_NPZ

In [47]:
def transform_df_NPZ(df):
    print("Apply strategies - NP: transform '4.2' Tab.")
    
    # PZT, N_NPZ-FSE, ÖW, N_NPF
    for c in df:
        c['Träger']=[]
        code_list=[ v for k, v in c.items() if k.startswith('Träger ')]
        n_bearbeitet_list=[ v for k, v in c.items() if k.startswith('n ')]
        for i in range(len(code_list)):
            if not math.isnan(n_bearbeitet_list[i]):
                c['Träger'].append({
                    'code':code_list[i],
                    'n_FSE,H':n_bearbeitet_list[i],
                    #'n':1
                })
        '''for a in D_PSE:
            for exc in a['exchanges']:
                for t in c['Träger']:
                    if exc['type']=='technosphere' and exc['code'] == t['code']:
                        t['n']=-exc['amount']'''
        entries_to_remove=[ k for k, v in c.items() if (k.startswith('Träger ') or k.startswith('n '))]
        for k in entries_to_remove:
            c.pop(k, None)
            
    for c in df:
        c['NPZ']=[]
        code_list=[ k for k, v in c.items() if k.startswith('NPF')]
        n_bearbeitet_list=[ v for k, v in c.items() if k.startswith('NPF')]
        for i in range(len(code_list)):
            if not math.isnan(n_bearbeitet_list[i]):
                c['NPZ'].append({
                    'code':code_list[i],
                    'n_NPZ,H':n_bearbeitet_list[i],
                })
        
    df_xPZ_v2=[]
    for PZ in df:
        PZ['ÖW']=[]
        if not any(d['Nr.'] == PZ['Nr.'] for d in df_xPZ_v2):
            df_xPZ_v2.append(PZ)
    for PZ1 in df:
        for PZ2 in df_xPZ_v2:
            if PZ1['Nr.']==PZ2['Nr.'] and PZ1['ÖK-Wert']!=0:
                PZ2['ÖW'].append({
                    'Nr.':PZ1['Ökologie-kriterium'],
                    'amount':PZ1['ÖK-Wert']
                })
    for PZ2 in df_xPZ_v2:
        del PZ2['Ökologie-kriterium']
        del PZ2['ÖK-Wert']
    
    return df_xPZ_v2

In [50]:
def create_ÖW_normiert(df):
    for PZ in df:
        for öw in PZ['ÖW']:
            öw['amount_normiert'] = öw['amount']/sum(item['n_FSE,H'] for item in PZ['Träger'])
    return df

# main.ipynb simulator

In [72]:
filepath = r"data/231105_LCA_v24_rsc_kf.xlsm"
sheet_names = [
    "1-Produktstruktur", 
    "2-Funktionsstruktur",
    "3-Ökologiekriterien",
    "4-Prozesse",
    "5-MTTFF MTBF",
    "6-Rückführungswahrs.",
    "7a-Transformbez_PF_gew",
    "7b-Transformbez_FP_gew",
    "8-Nutzungsprofile"
]

################################################################
df_PSE_v1, df_FSE_v1, df_ÖK, df_PZ_ges, df_MM, df_RW, T_PF, T_FP, df_NPF = read_excel_file(filepath=filepath, sheet_names=sheet_names)

df_FSE_v2 = unzip_dict(df_FSE_v1)
df_FSE_v3 = add_träger_column(df=df_FSE_v2)
D_FSE_v1 = transform_excel_sheet_to_matrix(df=df_FSE_v3)
D_FSE_v2 = sort_column_by_level(matrix=D_FSE_v1)

df_NPZ_v1 = filter_out_NPZ_PZ(df=df_PZ_ges)
df_NPZ_v2 = transform_df_NPZ(df=df_NPZ_v1) 
df_NPZ_v3 = create_ÖW_normiert(df=df_NPZ_v2) 

#df_NPF = read_excel_file(filepath=filepath, sheet_names=sheet_names)[8] #<--

#df_ÖK = read_excel_file(filepath=filepath, sheet_names=sheet_names)[2] #<--

Apply strategies: read excel-file ...
Apply strategies - NP: add täger-column ...
Apply strategies - NP: transform excel-sheet to D_FSE matrix ...
Apply strategies - NP: sort matrix-column by level (descending) ...
Apply strategies - NP: create '4.2' Tab.
Apply strategies - NP: transform '4.2' Tab.


In [73]:
df_NPZ_v3
#df_NPF
#D_FSE_v2

[{'Nr.': 'NP1001',
  'Prozess-schritt': 'Stanzen',
  'Ressource': 'M201830',
  'Dauer [min]': '55 min',
  'NPF001': 7.0,
  'NPF002': 4.0,
  'NPF003': 5.0,
  'Träger': [{'code': 'F01', 'n_FSE,H': 34}],
  'NPZ': [{'code': 'NPF001', 'n_NPZ,H': 7.0},
   {'code': 'NPF002', 'n_NPZ,H': 4.0},
   {'code': 'NPF003', 'n_NPZ,H': 5.0}],
  'ÖW': [{'Nr.': 'ÖK001',
    'amount': 5.2,
    'amount_normiert': 0.15294117647058825},
   {'Nr.': 'ÖK004', 'amount': 15.3, 'amount_normiert': 0.45},
   {'Nr.': 'ÖK005', 'amount': 0.3, 'amount_normiert': 0.008823529411764706},
   {'Nr.': 'ÖK006', 'amount': 0.18, 'amount_normiert': 0.005294117647058823}]},
 {'Nr.': 'NP1002',
  'Prozess-schritt': 'Maschinenaufstellung gewährleisten',
  'Ressource': 'M201830',
  'Dauer [min]': 2,
  'NPF001': 8.0,
  'NPF002': 6.0,
  'NPF003': 7.0,
  'Träger': [{'code': 'F02', 'n_FSE,H': 680}],
  'NPZ': [{'code': 'NPF001', 'n_NPZ,H': 8.0},
   {'code': 'NPF002', 'n_NPZ,H': 6.0},
   {'code': 'NPF003', 'n_NPZ,H': 7.0}],
  'ÖW': [{'Nr.': '

# Spielwiese

In [79]:
D_FSE=D_FSE_v2
df=df_NPZ_v3
df_2=df_NPF

for PZ in df:
    af_nenner_list=[]
    for NPF in df_2.values():
        af_nenner_list.append({
            'Nr.':NPF['Nr.'],
            'af_nenner':0
        })
    PZ['af_nenner']=af_nenner_list
    for PZT in PZ['Träger']:
        PZT['AF'] = []
        for n_NPZ in PZ['NPZ']:
            PZT['AF'].append({
                'code':n_NPZ['code'],
                'n_NPZ,H':n_NPZ['n_NPZ,H'],
                'af_zähler':(PZT['n_FSE,H']*n_NPZ['n_NPZ,H'])
            })
            for af_nenner in PZ['af_nenner']:
                if af_nenner['Nr.']==n_NPZ['code']:
                    af_nenner['af_nenner']+=(PZT['n_FSE,H']*n_NPZ['n_NPZ,H'])
    #print(PZ['af_nenner'])
    for PZT in PZ['Träger']:
        for AF in PZT['AF']:
            for af_nenner in PZ['af_nenner']:
                if af_nenner['Nr.']==AF['code']:
                    AF['af']=AF['af_zähler']/af_nenner['af_nenner']
        

df[2]

{'Nr.': 'NP1003',
 'Prozess-schritt': 'Sicheren Stand gewährleisten',
 'Ressource': 'M201830',
 'Dauer [min]': 1,
 'NPF001': 9.0,
 'NPF002': 8.0,
 'NPF003': 9.0,
 'Träger': [{'code': 'F03',
   'n_FSE,H': 680,
   'AF': [{'code': 'NPF001', 'n_NPZ,H': 9.0, 'af_zähler': 6120.0, 'af': 1.0},
    {'code': 'NPF002', 'n_NPZ,H': 8.0, 'af_zähler': 5440.0, 'af': 1.0},
    {'code': 'NPF003', 'n_NPZ,H': 9.0, 'af_zähler': 6120.0, 'af': 1.0}]}],
 'NPZ': [{'code': 'NPF001', 'n_NPZ,H': 9.0},
  {'code': 'NPF002', 'n_NPZ,H': 8.0},
  {'code': 'NPF003', 'n_NPZ,H': 9.0}],
 'ÖW': [{'Nr.': 'ÖK002',
   'amount': 0.2,
   'amount_normiert': 0.00029411764705882356},
  {'Nr.': 'ÖK006', 'amount': 0.14, 'amount_normiert': 0.00020588235294117648}],
 'af_nenner': [{'Nr.': 'NPF001', 'af_nenner': 6120.0},
  {'Nr.': 'NPF002', 'af_nenner': 5440.0},
  {'Nr.': 'NPF003', 'af_nenner': 6120.0}]}

In [67]:
df

[{'Nr.': 'NP1001',
  'Prozess-schritt': 'Stanzen',
  'Ressource': 'M201830',
  'Dauer [min]': '55 min',
  'NPF001': 7.0,
  'NPF002': 4.0,
  'NPF003': 5.0,
  'Träger': [{'code': 'F01', 'n_bearbeitet': 34}],
  'ÖW': [{'Nr.': 'ÖK001',
    'amount': 5.2,
    'amount_normiert': 0.15294117647058825},
   {'Nr.': 'ÖK004', 'amount': 15.3, 'amount_normiert': 0.45},
   {'Nr.': 'ÖK005', 'amount': 0.3, 'amount_normiert': 0.008823529411764706},
   {'Nr.': 'ÖK006', 'amount': 0.18, 'amount_normiert': 0.005294117647058823}],
  'n_NPZ': [{'code': 'NPF001', 'n_bearbeitet': 7.0},
   {'code': 'NPF002', 'n_bearbeitet': 4.0},
   {'code': 'NPF003', 'n_bearbeitet': 5.0}]},
 {'Nr.': 'NP1002',
  'Prozess-schritt': 'Maschinenaufstellung gewährleisten',
  'Ressource': 'M201830',
  'Dauer [min]': 2,
  'NPF001': 8.0,
  'NPF002': 6.0,
  'NPF003': 7.0,
  'Träger': [{'code': 'F02', 'n_bearbeitet': 680}],
  'ÖW': [{'Nr.': 'ÖK001',
    'amount': 1.2,
    'amount_normiert': 0.001764705882352941},
   {'Nr.': 'ÖK004', 'amou